# Getting Started

This notebook walks through the different types of guards you can use. Guards are a set of directives that can be used to restrict the output of agents, chains, prompts, or really any function that outputs a string. 

## @RestrictionGuard
RestrictionGuard is used to restrict output using an llm. By passing in a set of restrictions like "the output must be in latin" or "The output must be about baking" you can start to prevent your chain, agent, tool, or any llm generally from returning unpredictable content. 

In [1]:
from langchain.llms import OpenAI
from langchain.guards import RestrictionGuard

llm = OpenAI(temperature=0.9)

text = "What would be a good company name a company that makes colorful socks for romans?"

@RestrictionGuard(restrictions=['output must be in latin'], llm=llm, retries=0)
def sock_idea():
    return llm(text)
    
sock_idea()

The restriction guard works by taking in a set of restrictions, an llm to use to judge the output on those descriptions, and an int, retries, which defaults to zero and allows a function to be called again if it fails to pass the guard.

Restrictions should always be written in the form out 'the output must x' or 'the output must not x.'

In [ ]:
@RestrictionGuard(restrictions=['output must be about baking'], llm=llm, retries=1)
def baking_bot(user_input):
    return llm(user_input)
    
baking_bot(input("Ask me any question about baking!"))

The restriction guard works by taking your set of restrictions and prompting a provided llm to answer true or false whether a provided output violates those restrictions. Since it uses an llm, the results of the guard itself can be unpredictable. 

The restriction guard is good for moderation tasks that there are not other tools for, like moderating what type of content (baking, poetry, etc) or moderating what language.

The restriction guard is bad at things llms are bad at. For example, the restriction guard is bad at moderating things dependent on math or individual characters (no words greater than 3 syllables, no responses more than 5 words, no responses that include the letter e).

## @StringGuard

The string guard is used to restrict output that contains some percentage of a provided string. Common use cases may include preventing prompt leakage or preventing a list of derogatory words from being used. The string guard can also be used for things like preventing common outputs or preventing the use of protected words. 

The string guard takes a list of protected strings, a 'leniency' which is just the percent of a string that can show up before the guard is triggered (lower is more sensitive), and a number of retries.

Unlike the restriction guard, the string guard does not rely on an llm so using it is computationally cheap and fast.

For example, suppose we want to think of sock ideas but want unique names that don't already include the word 'sock':

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.guards import StringGuard
from langchain.chains import LLMChain

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

@StringGuard(protected_strings=['sock'], leniency=1, retries=5)
def sock_idea():
    return chain.run("colorful socks")
    
sock_idea()

If we later decided that the word 'fuzzy' was also too generic, we could add it to protected strings:

In [ ]:
@StringGuard(protected_strings=['sock', 'fuzzy'], leniency=1, retries=5)
def sock_idea():
    return chain.run("colorful socks")

*NB: Leniency is set to 1 for this example so that only strings that include the whole word "sock" will violate the guard.*

*NB: Capitalization does not count as a difference when checking differences in strings.*

Suppose that we want to let users ask for sock company names but are afraid they may steal out super secret genius sock company naming prompt. The first thought may be to just add our prompt template to the protected strings. The problem, though, is that the leniency for our last 'sock' guard is too high: the prompt may be returned a little bit different and not be caught if the guard leniency is set to 100%. The solution is to just add two guards! The sock one will be checked first and then the prompt one. This can be done since all a guard does is look at the output of the function below it.

For our prompt protecting string guard, we will set the leniency to 50%. If 50% of the prompt shows up in the answer, something probably went wrong!

In [ ]:
llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["description"],
    template="What is a good name for a company that makes {description} type of socks?",
)

chain = LLMChain(llm=llm, prompt=prompt)

@StringGuard(protected_strings=[prompt.template], leniency=.5, retries=5)
@StringGuard(protected_strings=['sock'], leniency=1, retries=5)
def sock_idea():
    return chain.run(input("What type of socks does your company make?"))

## @CustomGuard

The custom guard allows you to easily turn any function into your own guard! The custom guard takes in a function and, like other guards, a number of retries. The function should take a string as input and return True if the string violates the guard and False if not. 

One use cases for this guard could be to create your own local classifier model to, for example, classify text as "on topic" or "off topic." Or, you may have a model that determines sentiment. You could take these models and add them to a custom guard to ensure that the output of your llm, chain, or agent is exactly inline with what you want it to be.

Here's an example of a simple guard that prevents jokes from being returned that are too long.

In [ ]:
from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.guards import CustomGuard

llm = OpenAI(temperature=0.9)

prompt_template = "Tell me a {adjective} joke"
prompt = PromptTemplate(
    input_variables=["adjective"], template=prompt_template
)
chain = LLMChain(llm=OpenAI(), prompt=prompt)

def is_long(llm_output):
    return len(llm_output) > 100

@CustomGuard(guard_function=is_long, retries=1)
def call_chain():
    return chain.run(adjective="political")

call_chain()